# Social Network Analysis: Lab Exercise 3
## Full name: *Andreas Kontogiannis*
### Student NTUA Username: el15187
#### ECE, NTUA, 2019-2020

## Εισαγωγή
<h3>Γενετικοί Αλγόριθμοι</h3>
<p style=text-align:justify>Στην παρούσα άσκηση θα εξοικειωθούμε με την χρήση των γενετικών αλγορίθμων και την επιλογή των παραμέτρων τους. Επιπλέον, θα αναπτύξουμε ένα γενετικό αλγόριθμο για τον εντοπισμό κοινοτήτων σε πραγματικά δίκτυα και θα συγκρίνουμε το αποτέλεσμά του με τους αλγόριθμους εντοπισμού κοινοτήτων που εφαρμόστηκαν στην 2<sup>η</sup> εργαστηριακή άσκηση.</p>

Θα χρησιμοποιήσουμε τα παρακάτω modules της python3.

In [1]:
%matplotlib inline

import matplotlib
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

### Άσκηση 1: Εξοικείωση με Γενετικούς Αλγόριθμους

### Εκφώνηση

#### Πρόβλημα ONEMAX

<p style=text-align:justify>Βρείτε την δυαδική ακολουθία $(x_1, x_2, ..., x_n)$ που μεγιστοποιεί το άθροισμα $x_1 + x_2 + ... + x_n, n=20$. Αναλύστε την επίδραση του μεγέθους του πληθυσμού, της πιθανότητας διασταύρωσης (crossover) και της πιθανότητας μετάλλαξης (mutation) στην ποιότητα της λύσης (πόσο κοντά είναι στην προφανή βέλτιστη λύση) που προκύπτει από το γενετικό αλγόριθμο. Συγκεκριμένα, μεταβάλλετε τον πληθυσμό από $10$ ως $100$ χρωμοσώματα με βήμα $10$,την πιθανότητα διασταύρωσης από $0.3$ ως $0.9$ με βήμα $0.1$ και την πιθανότητα μετάλλαξης από $0.01$ ως $0.2$ με βήμα $0.01$. Εξετάστε την ποιότητα της λύσης του γενετικού αλγορίθμου καθώς αλλάζετε τις παραμέτρους. Ελιτισμό μπορείτε να χρησιμοποιήσετε αν το επιθυμείτε.</p>

### Απάντηση

Θα χρησιμοποιήσουμε για fitness το άθροισμα των όρων της ακολουθίας, δεδομένου ότι το καλύτερο θα είναι η τιμή 20, δηλαδή όλοι οι όροι να είναι άσσοι.

**Γενετικός Αλγόριθμος**

In [30]:
from numpy import sum, zeros, reshape, arange
import random


def create_population(m=10, n=20):
    # Create the initial population, a binary sequence of size n
    # OneMax problem
    return np.random.randint(2, size=(m,n))
    

def find_fitness(i, seq):
    # Calculate fitness of chromosomes
    return sum(seq[i])
    

def proportional_selection(seq, elitism):
    # Select chromosomes to pass to next population proportionally to their fitness
    m = seq.shape[0]
    n = seq.shape[1]
    new_seq = zeros((m,n))
    
    fitness = []
    for i in range(m):
        fitness.append(find_fitness(i, seq))    
    sum_fitness = sum(fitness)
    
    # The first x fittest pass to next population -- elitism
    x_indexes = argsort(fitness)[::-1][0:elitism]
    new_seq[0:elitism] = seq[x_indexes]
    
    # Roulette
    for i in range(elitism, m):
        rand = random.uniform(0,1) # Random[0,1]
        k = 0
        summ = fitness[k] / sum_fitness
        while(k < m - 1 and rand > summ): 
            k += 1
            summ += fitness[k] / sum_fitness 

        new_seq[i,:] = reshape(seq[k,:], (1,n))
        
    return new_seq  
        

def crossover(Pc, new_seq):
    # Apply crossover to population
    m = new_seq.shape[0]
    n = new_seq.shape[1]
    
    i = 0
    while (i < m - 1):
        if (random.uniform(0, 1)) <= Pc:
            pos = np.random.randint(n-1)
            
            k = pos + 1
            while (k < n):
                aux = new_seq[i][k]
                new_seq[i][k] = new_seq[i+1][k]
                new_seq[i+1][k] = aux
                
                k += 1
            
        i += 2
    
    return new_seq


def mutation(Pm, new_seq):
    # Apply mutation to population
    m = new_seq.shape[0]
    n = new_seq.shape[1]
    
    for i in range(m):
        for k in range(n):
            if (random.uniform(0, 1)) < Pm:
                new_seq[i][k] = (new_seq[i][k] + 1) % 2  # invert
    
    return new_seq


def genetic_algorithm(Pm, Pc, m, elitism, n=20, T=30):
    # Genetic algorithm
    new_seq = create_population(m,n)
    for t in range(T):
        new_seq = proportional_selection(new_seq, elitism)
        new_seq = crossover(Pc, new_seq)
        new_seq = mutation(Pm, new_seq)
    
    return new_seq

Τρέχουμε τον γενετικό αλγόριθμο με τις ζητούμενες υπερπαραμέτρους και επιλέγουμε εκείνους τους συνδυασμούς που δίνουν το καλύτερο fitness.

In [31]:
from numpy import max, argsort

# main
max_fit = []
params = []
for m in range(10, 110, 10):
    for pc in arange(0.3, 1, 0.1):
        for pm in arange(0.01, 0.21, 0.01):
            for elitism in range(0, 3):
                seq = genetic_algorithm(pm, pc, m, elitism, n=20, T=30)
                fitness = [find_fitness(i, seq) for i in range(m)]
                max_fit.append(max(fitness))
                params.append("m={}, Pc={}, Pm={}, elitism={}, fitness=".format(m, round(pc,3), 
                                                                                 round(pm,2), elitism))
            
indexes = argsort(max_fit)[::-1]
sort = sorted(max_fit)[::-1]
for i, fit in enumerate(sort):
    print(params[indexes[i]] + str(fit))
print()
print("Max fitness: ", max(max_fit))

m=30, Pc=0.6, Pm=0.03, elitism=1, fitness=20.0
m=30, Pc=0.9, Pm=0.01, elitism=2, fitness=20.0
m=100, Pc=0.5, Pm=0.04, elitism=2, fitness=20.0
m=90, Pc=0.6, Pm=0.04, elitism=2, fitness=20.0
m=100, Pc=0.5, Pm=0.07, elitism=1, fitness=20.0
m=30, Pc=0.4, Pm=0.01, elitism=2, fitness=20.0
m=10, Pc=0.6, Pm=0.04, elitism=1, fitness=20.0
m=90, Pc=0.6, Pm=0.02, elitism=2, fitness=20.0
m=90, Pc=0.6, Pm=0.02, elitism=1, fitness=20.0
m=10, Pc=0.6, Pm=0.03, elitism=2, fitness=20.0
m=40, Pc=0.7, Pm=0.02, elitism=2, fitness=20.0
m=30, Pc=0.4, Pm=0.03, elitism=1, fitness=20.0
m=90, Pc=0.6, Pm=0.01, elitism=2, fitness=20.0
m=90, Pc=0.6, Pm=0.01, elitism=1, fitness=20.0
m=30, Pc=0.9, Pm=0.02, elitism=2, fitness=20.0
m=50, Pc=0.8, Pm=0.01, elitism=1, fitness=20.0
m=50, Pc=0.8, Pm=0.01, elitism=2, fitness=20.0
m=80, Pc=0.5, Pm=0.01, elitism=2, fitness=20.0
m=50, Pc=0.8, Pm=0.02, elitism=1, fitness=20.0
m=50, Pc=0.8, Pm=0.02, elitism=2, fitness=20.0
m=50, Pc=0.4, Pm=0.02, elitism=2, fitness=20.0
m=20, Pc=0.

m=20, Pc=0.9, Pm=0.03, elitism=0, fitness=16.0
m=20, Pc=0.3, Pm=0.05, elitism=0, fitness=16.0
m=20, Pc=0.9, Pm=0.02, elitism=0, fitness=16.0
m=20, Pc=0.3, Pm=0.06, elitism=0, fitness=16.0
m=20, Pc=0.3, Pm=0.07, elitism=0, fitness=16.0
m=20, Pc=0.3, Pm=0.07, elitism=2, fitness=16.0
m=30, Pc=0.8, Pm=0.08, elitism=0, fitness=16.0
m=40, Pc=0.4, Pm=0.12, elitism=1, fitness=16.0
m=30, Pc=0.8, Pm=0.08, elitism=1, fitness=16.0
m=20, Pc=0.3, Pm=0.08, elitism=2, fitness=16.0
m=20, Pc=0.3, Pm=0.09, elitism=2, fitness=16.0
m=20, Pc=0.8, Pm=0.2, elitism=2, fitness=16.0
m=20, Pc=0.3, Pm=0.11, elitism=0, fitness=16.0
m=40, Pc=0.4, Pm=0.11, elitism=1, fitness=16.0
m=20, Pc=0.8, Pm=0.18, elitism=2, fitness=16.0
m=20, Pc=0.3, Pm=0.12, elitism=1, fitness=16.0
m=40, Pc=0.4, Pm=0.11, elitism=0, fitness=16.0
m=20, Pc=0.3, Pm=0.17, elitism=1, fitness=16.0
m=20, Pc=0.4, Pm=0.01, elitism=0, fitness=16.0
m=20, Pc=0.8, Pm=0.17, elitism=2, fitness=16.0
m=40, Pc=0.4, Pm=0.16, elitism=2, fitness=16.0
m=20, Pc=0.9, 

Max fitness:  20.0


Βλέπουμε ότι το καλύτερο σκορ, 20.0, ταυτίζεται με το βέλτιστο σκορ, δηλαδή όλα τα ψηφία της ακολουθίας να είναι άσσοι. Παρατηρούμε αρκετούς συνδυασμούς παραμέτρων που δίνουν το βέλτιστο σκορ. Αυτο συμβαίνει στην περίπτωση του ελιτισμού. Έχουμε τρέξει τον αλγόριθμο, αρκετές φορές, χωρίς την χρήση ελιτισμού, και παρατηρήσαμε μία έως καμία απάντηση που δίνει το βέλτιστο σκορ. Στην περίπτωση του ελιτισμού, παρατηρούμε πως για τα μεγαλύτερα σκορ, οι τιμές  αυτού που προτιμούνται είναι συνήθως οι μεγαλύτερες, με αποτέλεσμα να φτάνουμε στο συμπέρασμα πως η αύξηση του  τείνει να ευνοεί την αύξηση του fitness. Για τις υπόλοιπες παραμετρους, δεδομένου ότι αυτές είναι οι πιο συνήθεις παράμετροι που επιλέγονται, μπορούμε να πούμε πως οι τιμές που προτιμούνται για το $P_m$ τείνουν να είναι οι πολύ μικρές, δηλαδή μικρότερες από 0.1, ενώ για το $P_c$ μεγαλύτερες από 0.5, με τις τιμές 0.7, 0.8 να είναι προτιμηταίες για όλα τα μεγάλα σκορ (18, 19, 20).

### Άσκηση 2: Εντοπισμός Κοινοτήτων σε Γράφους Κοινωνικών Δικτύων με Χρήση Γενετικών Αλγορίθμων

### Εκφώνηση

<p style=text-align:justify>Στην παρούσα άσκηση θα αναπτύξετε κώδικα για εντοπισμό κοινοτήτων σε γράφους δικτύων με χρήση γενετικών αλγορίθμων. Ακολούθως θα εφαρμόσετε το γενετικό αλγόριθμο για τον εντοπισμό κοινοτήτων σε πραγματικά δίκτυα και θα συγκρίνετε το αποτέλεσμά του με εκείνο των αλγορίθμων της δεύτερης εργαστηριακής άσκησης χρησιμοποιώντας τη μετρική modularity.</p>

##### Αναπαράσταση υποψήφιων λύσεων (χρωμοσωμάτων)
<p style=text-align:justify>Κάθε χρωμόσωμα $b_{i,t}$ του πληθυσμού τη χρονική στιγμή $t$ είναι ένα διάνυσμα με μήκος ίσο με το πλήθος των κόμβων του δικτύου, $n$. Αν $b_{i,t}(j)=d$ τότε οι κόμβοι $j,d$ ανήκουν στην ίδια κοινότητα. Επιπλέον για να ισχύει $b_{i,t}(j)=d$ , οι κόμβοι $j,d$ θα πρέπει να συνδέονται στην αρχική τοπολογία. Τότε, βρίσκοντας τις συνδεδεμένες συνιστώσες του χρωμοσώματος $b_{i,t}$ , προκύπτουν οι κοινότητες του δικτύου.</p>

##### Αρχικοποίηση
<p style=text-align:justify>Η αρχική γενιά του πληθυσμού θα πρέπει να αποτελείται από χρωμοσώματα τα οποία θα είναι τυχαία επιλεγμένα και διορθωμένα ώστε ο αριθμός που υπάρχει σε κάθε θέση να είναι γείτονας του αντίστοιχου κόμβου. (Αν δεν ισχύει θα πρέπει να αντικατασταθεί από ένα γείτονα του αντίστοιχου κόμβου.) Θεωρείστε πληθυσμό $300$ χρωμοσωμάτων.</p>

##### Συνάρτηση Fitness
<p style=text-align:justify>Η συνάρτηση fitness υπολογίζεται όπως στις διαφάνειες 28 και 29 της αντίστοιχης διάλεξης του μαθήματος. Προσοχή για κάθε συνιστώσα (component) του γράφου θα πρέπει να βρίσκετε τον αντίστοιχο υπογράφο.</p>

##### Επιλογή (Selection)
<p style=text-align:justify>Η επιλογή χρωμοσωμάτων από μία γενιά του πληθυσμού για την κατασκευή της επόμενης γενιάς γίνεται με τη μέθοδο της ρουλέτας (διαφάνειες μαθήματος και αλγόριθμος). Τροποποιήστε τον παραπάνω αλγόριθμο ώστε να εφαρμόσετε και ελιτισμό, δηλαδή τα $x$ πρώτα χρωμοσώματα της νέας γενιάς θα είναι εκείνα τα χρωμοσώματα της προηγούμενης γενιάς που έχουν επιτύχει τη μέγιστη τιμή της συνάρτησης fitness.</p>

##### Διασταύρωση (Crossover)
<p style=text-align:justify>Η διασταύρωση πραγματοποιείται όπως περιγράφεται στον αλγόριθμο, μεταξύ διαδοχικών χρωμοσωμάτων της εκάστοτε γενιάς. Η πιθανότητα διασταύρωσης ανά ζεύγος είναι ίση με $p_C$.</p>

##### Μετάλλαξη (Mutation)
<p style=text-align:justify>Η μετάλλαξη πραγματοποιείται σε κάθε θέση του χρωμοσώματος με πιθανότητα $p_M$, όπως στον αλγόριθμο. Θα πρέπει ωστόσο ο αλγόριθμος να τροποποιηθεί ώστε να δίνει ένα τυχαίο γείτονα του κόμβου που αντιστοιχεί σε κάθε θέση ως αποτέλεσμα της μετάλλαξης.</p>

##### Κριτήριο Τερματισμού (Stopping Criterion)
<p style=text-align:justify>Ως κριτήριο τερματισμού ορίστε ένα μέγιστο πλήθος επαναλήψεων εφαρμογής του αλγορίθμου (γενεών πληθυσμού), ίσο με $30$, ή ότι η βέλτιστη τιμή της συνάρτησης fitness δεν έχει μεταβληθεί για τις τελευταίες $5$ επαναλήψεις. Εφαρμόστε τον παραπάνω γενετικό αλγόριθμο στις πραγματικές τοπολογίες της δεύτερης εργαστηριακής άσκησης που δίνονται και στον Πίνακα 1. Οι παράμετροι που θα πρέπει να εξεταστούν δίνονται στον παρακάτω πίνακα. Προσοχή θα πρέπει να εξεταστούν όλοι οι συνδυασμοί.</p>

<table style=width:100%>
<tr>
    <th style=text-align:left>Παράμετρος</th>
    <th style=text-align:left>Τιμές</th>
</tr>
<tr>
    <td style=text-align:left>Διαστάυρωση $p_C$</td>
    <td style=text-align:left>$0.7:0.9$ με βήμα $0.1$</td>
</tr>
<tr>
    <td style=text-align:left>Μετάλλαξη $p_M$</td>
    <td style=text-align:left>$\{0.1,0.2\}$</td>
</tr>
<tr>
    <td style=text-align:left>Ελιτισμός $x$</td>
    <td style=text-align:left>$1:3$ με βήμα $1$</td>
</tr>
</table>

<p style=text-align:justify>Συγκρίνετε το αποτέλεσμα του γενετικού αλγορίθμου (το καλύτερο για τις διάφορες επιλογές παραμέτρων) με εκείνο των αλγορίθμων εντοπισμού κοινοτήτων της εργαστηριακής άσκησης 2 (δίνονται και στον Πίνακα 2), υπολογίζοντας το modularity. Θα πρέπει να έχετε πρωτίστως μετατρέψει το αποτέλεσμα του γενετικού αλγορίθμου σε κατάλληλη μορφη πάνω στο χρωμόσωμα που επιτυγχάνει τη βέλτιστη τιμή της συνάρτησης fitness.</p>


### Απάντηση

Προεπεξεργαζόμαστε αρχικά τα δεδομένα, προκειμένου να αναπαριστούν μεταβλητές του networkx, όπως κάναμε και στην 2η εργαστηριακή άσκηση.

In [3]:
# Import real topologies 
football_gml = nx.read_gml("./football.gml")
lesmis_gml = nx.read_gml("./lesmis.gml")
dolphins_gml = nx.read_gml("./dolphins.gml")

def conv2int(G, start_value):
    nG = nx.convert_node_labels_to_integers(G, first_label=start_value)
    G_mapping = dict(enumerate(G.nodes, 0))
    return nG, G_mapping

football, football_map = conv2int(football_gml, 0)
lesmis, lesmis_map = conv2int(lesmis_gml, 0)
dolphins, dolphins_map = conv2int(dolphins_gml, 0)

**Γενετικός Αλγόριθμος**

In [17]:
from numpy import mean, argsort, zeros, reshape, squeeze, asarray
import numpy as np
import random


def create_population(G, m=300):
    # Create the initial population
    '''
        G:   networkx Graph
        m:   int, num of chromosomes
    '''
    n = len(list(G.nodes()))
    chrom = zeros((m,n))   
    
    # Memorization of neighbors 
    neighbors = []
    for i in range(n):
        neighbors.append(list(G.neighbors(i)))
            
    for i in range(m):
        for j in range(n):
            chrom[i][j] = np.random.choice(neighbors[j], 1)[0]
    
    return chrom


def find_fitness(chrom, idx, graph):
    # Calculate fitness of chromosomes
    '''
        chrom:   array of chromosomes
        i:       int, index of chromosome that we want to find its fitness
        Graph:   networkx Graph, initial graph    
    '''
        
    # Initialize chromosomes graph
    G = nx.Graph()
    for j in range(chrom.shape[1]):
        G.add_node(j)
    for v,u in enumerate(chrom[idx,:]):
        if G.has_edge(v,u):
            continue
        G.add_edge(v,u)
    
    # Adjacency matrix of initial graph  
    A = nx.to_numpy_matrix(graph)
    A = squeeze(asarray(A))  # numpy array
            
    # Connected components - communities of chromosomes graph
    comps = list(nx.connected_components(G))
    num_comps = len(comps)
    
    fitness = 0
    for comp in comps:
        ll = list(comp)
        for i,item in enumerate(ll):
            ll[i] = int(item)

        # Adjacency matrix of subgraph
        S = A[ll][:,ll]

        # rows mean values of subgraph
        a = mean(S, axis=1)

        # M(S)
        # r=1
        I = S.shape[0]
        Ms = sum(a) / I
    
        # u_s 
        u_s = sum(sum(S))
    
        # Qs
        Qs = Ms * u_s
        fitness += Qs
            
    return fitness


def proportional_selection(chrom, x, G):
    # Select chromosomes to pass to next population proportionally to their fitness
    '''
        chrom:   array of chromosomes
        x:       int, value of elitism
        G:       networkx graph
    '''

    m = chrom.shape[0]
    n = chrom.shape[1]
    new_chrom = zeros((m,n))
    
    fitness = []
    for i in range(m):
        fitness.append(find_fitness(chrom, i, G))
        
    sum_fitness = sum(fitness)
    
    # The first x fittest pass to next population -- elitism
    x_indexes = argsort(fitness)[::-1][0:x]
    new_chrom[0:x] = chrom[x_indexes]
        
    # Roulette
    for i in range(x, m):
        rand = random.uniform(0,1) # Random[0,1]
        k = 0
        summ = fitness[k] / sum_fitness
        while(k < m - 1 and rand > summ): 
            k += 1
            summ += fitness[k] / sum_fitness 

        new_chrom[i,:] = reshape(chrom[k,:], (1,n))
        
    return new_chrom  
    

def crossover(Pc, chrom):
    '''
        Pc:          float, crossover probability
        chrom:       array of chromosomes, parents
    '''
    # Apply crossover to population
    m = chrom.shape[0]
    n = chrom.shape[1]
    
    count = 0
    new_chrom = zeros((m,n))  # child
    while (count < m):
        i = 0
        while (i < m - 1):
            x = random.uniform(0, 1)
            if x <= Pc:            
                # Mask
                mask = np.random.randint(2, size=n)

                # Child
                for j in range(n):
                    if mask[j] == 0:
                        new_chrom[count][j] = chrom[i][j]

                    elif mask[j] == 1:
                        new_chrom[count][j] = chrom[i+1][j]
                 
                # Next child
                count += 1
                if count == m:
                    return new_chrom       
            
            i += 1 
        
    
    return new_chrom


def mutation(G, Pm, new_chrom):
    # Apply mutation to population
    '''
        G:           networkx Graph
        Pm:          float, crossover probability
        new_chrom:   array of chromosomes
    '''
    m = new_chrom.shape[0]
    n = new_chrom.shape[1]
    
    # Memorization of neighbors 
    neighbors = []
    for i in range(n):
        neighbors.append(list(G.neighbors(i)))
    
    for i in range(m):
        for k in range(n):
            if (random.uniform(0, 1)) < Pm:
                new_chrom[i][k] = np.random.choice(neighbors[k], 1)[0] # mutate
    
    return new_chrom


def genetic_algorithm(G, Pm, Pc, m, x, T=30):
    # Genetic algorithm
    new_chrom = create_population(G, m)
    best_fit = 0
    count_fit = 0    
    for t in range(T):
        new_chrom = proportional_selection(new_chrom, x, G)
        new_chrom = crossover(Pc, new_chrom)
        new_chrom = mutation(G, Pm, new_chrom)
    
        # Stopping Criterion
        fitness = [] 
        for idx in range(len(new_chrom)):
             fitness.append(find_fitness(new_chrom, idx, G))                
        cur_fit = max(fitness)
        if cur_fit == best_fit:
            count_fit += 1   
        if count_fit == 5:
            return new_chrom
    
    return new_chrom

Τρέχουμε τον γενετικό αλγόριθμο με τις ζητούμενες υπερπαραμέτρους και επιλέγουμε εκείνους τους συνδυασμούς που δίνουν το καλύτερο fitness.

In [15]:
from numpy import max, arange, argmax

def runGen(G):
    m = 300
    best_chrom = []
    parameters = []
    for pc in arange(0.7, 0.9, 0.1):
        for pm in arange(0.1, 0.3, 0.1):
            for x in range(1, 4):        
                chrom = genetic_algorithm(G, pm, pc, m, x, T=30)
                fitness = [find_fitness(chrom, i, G) for i in range(m)]
                best_chrom.append(chrom[argmax(fitness)])
                parameters.append("Pc={}, Pm={}, x={}, best fitness={}".format(round(pc,3), 
                                                                               round(pm,2), x, max(fitness)))
                print("Pc={}, Pm={}, x={}, best fitness={}".format(round(pc,3), round(pm,2), x, max(fitness)))                
    
    return best_chrom, parameters

Παρακάτω, υλοποιούμε συνάρτηση που τυπώνει τα καλύτερα fitness για κάθε συνδυασμό και στην συνέχεια επιστρέφει το καλύτερο fitness που έχει δει.

In [25]:
# modularity for GA community detection algorithm 
def compute_modularity(G, best_chrom, parameters):
    modularities = []
    for i,chrom in enumerate(best_chrom):
        # Initialize chromosome graph
        GG = nx.Graph()
        for j in range(len(chrom)):
            GG.add_node(j)
        for v,u in enumerate(chrom):
            if GG.has_edge(u,v):
                continue
            GG.add_edge(v,u)

        # Connected components - communities of chromosomes graph
        comps = list(nx.connected_components(GG)) 
        communities = [list(comp) for comp in comps]
        
        # Modularity score
        modul = nx.algorithms.community.quality.performance(G, communities)
        modularities.append(modul)
        print(parameters[i] + ", modularity={}".format(modul))
        
    best_modularity_index = argmax(modularities)
    best_modularity = modularities[best_modularity_index]
    best_params = parameters[best_modularity_index]
    print(best_params + ", best modularity={}".format(best_modularity))
    
    return best_modularity

Τρέχουμε τον γενετικό αλγόριθμο για τις τρεις τοπολογίες που μας δόθηκαν και υπολογίζουμε για καθεμία το καλύτερο modularity, τυπώνοντάς το στο τέλος μαζί με τις αντίστοιχες παραμέτρους του.

##### Football results

In [18]:
football_chrom, football_param = runGen(football)

Pc=0.7, Pm=0.1, x=1, best fitness=305.6169876363738
Pc=0.7, Pm=0.1, x=2, best fitness=283.76490348947516
Pc=0.7, Pm=0.1, x=3, best fitness=335.1133269616711
Pc=0.7, Pm=0.2, x=1, best fitness=299.0099698289217
Pc=0.7, Pm=0.2, x=2, best fitness=265.8672055357184
Pc=0.7, Pm=0.2, x=3, best fitness=264.92315312253515
Pc=0.8, Pm=0.1, x=1, best fitness=292.16431308362604
Pc=0.8, Pm=0.1, x=2, best fitness=288.68268034796546
Pc=0.8, Pm=0.1, x=3, best fitness=320.15297032434046
Pc=0.8, Pm=0.2, x=1, best fitness=268.02545113777484
Pc=0.8, Pm=0.2, x=2, best fitness=278.57348532451107
Pc=0.8, Pm=0.2, x=3, best fitness=286.8214101239669
Pc=0.9, Pm=0.1, x=1, best fitness=277.6751347946586
Pc=0.9, Pm=0.1, x=2, best fitness=338.7194079858548
Pc=0.9, Pm=0.1, x=3, best fitness=326.66265132965805
Pc=0.9, Pm=0.2, x=1, best fitness=268.06866877127413
Pc=0.9, Pm=0.2, x=2, best fitness=315.26746048133
Pc=0.9, Pm=0.2, x=3, best fitness=308.4509858927841


In [27]:
foot_modul = compute_modularity(football, football_chrom, football_param)
foot_modul

Pc=0.7, Pm=0.1, x=1, best fitness=305.6169876363738, modularity=0.893211289092296
Pc=0.7, Pm=0.1, x=2, best fitness=283.76490348947516, modularity=0.8742944317315027
Pc=0.7, Pm=0.1, x=3, best fitness=335.1133269616711, modularity=0.8825324180015256
Pc=0.7, Pm=0.2, x=1, best fitness=299.0099698289217, modularity=0.8874141876430206
Pc=0.7, Pm=0.2, x=2, best fitness=265.8672055357184, modularity=0.8842105263157894
Pc=0.7, Pm=0.2, x=3, best fitness=264.92315312253515, modularity=0.8486651411136537
Pc=0.8, Pm=0.1, x=1, best fitness=292.16431308362604, modularity=0.887719298245614
Pc=0.8, Pm=0.1, x=2, best fitness=288.68268034796546, modularity=0.8825324180015256
Pc=0.8, Pm=0.1, x=3, best fitness=320.15297032434046, modularity=0.905720823798627
Pc=0.8, Pm=0.2, x=1, best fitness=268.02545113777484, modularity=0.9067887109077041
Pc=0.8, Pm=0.2, x=2, best fitness=278.57348532451107, modularity=0.8706331045003813
Pc=0.8, Pm=0.2, x=3, best fitness=286.8214101239669, modularity=0.7938977879481312


0.9067887109077041

##### Lesmis results

In [28]:
lesmis_chrom, lesmis_param = runGen(lesmis)

Pc=0.7, Pm=0.1, x=1, best fitness=197.78149504132233
Pc=0.7, Pm=0.1, x=2, best fitness=191.5858777777778
Pc=0.7, Pm=0.1, x=3, best fitness=196.54408400615148
Pc=0.7, Pm=0.2, x=1, best fitness=182.17915451856123
Pc=0.7, Pm=0.2, x=2, best fitness=180.39044093466515
Pc=0.7, Pm=0.2, x=3, best fitness=167.1957121698164
Pc=0.8, Pm=0.1, x=1, best fitness=198.69803379280987
Pc=0.8, Pm=0.1, x=2, best fitness=205.86198875739643
Pc=0.8, Pm=0.1, x=3, best fitness=202.62212474432027
Pc=0.8, Pm=0.2, x=1, best fitness=174.88000000000002
Pc=0.8, Pm=0.2, x=2, best fitness=168.51158521440573
Pc=0.8, Pm=0.2, x=3, best fitness=182.12499999999997
Pc=0.9, Pm=0.1, x=1, best fitness=177.83984310357937
Pc=0.9, Pm=0.1, x=2, best fitness=188.63846371882084
Pc=0.9, Pm=0.1, x=3, best fitness=190.09969756738985
Pc=0.9, Pm=0.2, x=1, best fitness=174.8893827160494
Pc=0.9, Pm=0.2, x=2, best fitness=173.53380177514794
Pc=0.9, Pm=0.2, x=3, best fitness=190.46097505668934


In [29]:
lesmis_modul = compute_modularity(lesmis, lesmis_chrom, lesmis_param)
lesmis_modul

Pc=0.7, Pm=0.1, x=1, best fitness=197.78149504132233, modularity=0.867053998632946
Pc=0.7, Pm=0.1, x=2, best fitness=191.5858777777778, modularity=0.8701298701298701
Pc=0.7, Pm=0.1, x=3, best fitness=196.54408400615148, modularity=0.8991797676008202
Pc=0.7, Pm=0.2, x=1, best fitness=182.17915451856123, modularity=0.9036226930963773
Pc=0.7, Pm=0.2, x=2, best fitness=180.39044093466515, modularity=0.8643198906356802
Pc=0.7, Pm=0.2, x=3, best fitness=167.1957121698164, modularity=0.8732057416267942
Pc=0.8, Pm=0.1, x=1, best fitness=198.69803379280987, modularity=0.849624060150376
Pc=0.8, Pm=0.1, x=2, best fitness=205.86198875739643, modularity=0.8602187286397812
Pc=0.8, Pm=0.1, x=3, best fitness=202.62212474432027, modularity=0.9019138755980861
Pc=0.8, Pm=0.2, x=1, best fitness=174.88000000000002, modularity=0.8243335611756665
Pc=0.8, Pm=0.2, x=2, best fitness=168.51158521440573, modularity=0.8711551606288448
Pc=0.8, Pm=0.2, x=3, best fitness=182.12499999999997, modularity=0.7388926862611

0.9036226930963773

##### Dolphins results

In [30]:
dolphins_chrom, dolphins_param = runGen(dolphins)

Pc=0.7, Pm=0.1, x=1, best fitness=63.473560493827165
Pc=0.7, Pm=0.1, x=2, best fitness=62.93361443751054
Pc=0.7, Pm=0.1, x=3, best fitness=70.80881950821345
Pc=0.7, Pm=0.2, x=1, best fitness=68.46088435374148
Pc=0.7, Pm=0.2, x=2, best fitness=61.16836734693877
Pc=0.7, Pm=0.2, x=3, best fitness=66.28820357772739
Pc=0.8, Pm=0.1, x=1, best fitness=63.62271604938272
Pc=0.8, Pm=0.1, x=2, best fitness=69.44068495717846
Pc=0.8, Pm=0.1, x=3, best fitness=69.90316659170554
Pc=0.8, Pm=0.2, x=1, best fitness=65.00455924036281
Pc=0.8, Pm=0.2, x=2, best fitness=70.14215284154679
Pc=0.8, Pm=0.2, x=3, best fitness=65.58449987402368
Pc=0.9, Pm=0.1, x=1, best fitness=66.00845804988663
Pc=0.9, Pm=0.1, x=2, best fitness=66.49650793650794
Pc=0.9, Pm=0.1, x=3, best fitness=68.16420097450744
Pc=0.9, Pm=0.2, x=1, best fitness=65.62148193714378
Pc=0.9, Pm=0.2, x=2, best fitness=65.86372378980771
Pc=0.9, Pm=0.2, x=3, best fitness=65.05613463032272


In [31]:
dolphins_modul = compute_modularity(dolphins, dolphins_chrom, dolphins_param)
dolphins_modul

Pc=0.7, Pm=0.1, x=1, best fitness=63.473560493827165, modularity=0.7657324167107351
Pc=0.7, Pm=0.1, x=2, best fitness=62.93361443751054, modularity=0.866737176097303
Pc=0.7, Pm=0.1, x=3, best fitness=70.80881950821345, modularity=0.8794288736118456
Pc=0.7, Pm=0.2, x=1, best fitness=68.46088435374148, modularity=0.9042834479111581
Pc=0.7, Pm=0.2, x=2, best fitness=61.16836734693877, modularity=0.8662083553675304
Pc=0.7, Pm=0.2, x=3, best fitness=66.28820357772739, modularity=0.8619777895293496
Pc=0.8, Pm=0.1, x=1, best fitness=63.62271604938272, modularity=0.8741406663141195
Pc=0.8, Pm=0.1, x=2, best fitness=69.44068495717846, modularity=0.8387096774193549
Pc=0.8, Pm=0.1, x=3, best fitness=69.90316659170554, modularity=0.8741406663141195
Pc=0.8, Pm=0.2, x=1, best fitness=65.00455924036281, modularity=0.884188260179799
Pc=0.8, Pm=0.2, x=2, best fitness=70.14215284154679, modularity=0.8688524590163934
Pc=0.8, Pm=0.2, x=3, best fitness=65.58449987402368, modularity=0.8947646747752512
Pc=0.

0.9042834479111581

#### Σχόλια για τα αποτελέσματα του γενετικού αλγόριθμου ανάλογα με τις τιμές των παραμέτρων

### Find the best partition with Girvan-Newman, Spectral Clustering and Modularity Maximization
<p style=text-align:justify>Aποτέλεσματα των αλγορίθμων της δεύτερης εργαστηριακής άσκησης

In [4]:
import itertools

# Girvan_newman:partitions of at most k communities, select the one with the highest value of modularity.
def comd_GN(G,k):
    gn_G = nx.algorithms.community.centrality.girvan_newman(G)
    limited = itertools.takewhile(lambda c: len(c) <= k, gn_G)
    per_G = []
    com_G = []
    for communities in limited:
        per = nx.algorithms.community.quality.performance(G,communities)
        per_G.append(per) #list of performance of each partition
        com_G.append(communities) # list of partitions
    max_mod_G = max(per_G) # maximum modularity value
    index_G = per_G.index(max(per_G)) # index of maximum modularity
    best_G = com_G[index_G] # the partition of maximum modularity  
    num_best_G = len(best_G)
    return best_G, max_mod_G, num_best_G

GN_best_football, GN_mod_football, GN_num_football = comd_GN(football,50)
GN_best_lesmis, GN_mod_lesmis, GN_num_lesmis = comd_GN(lesmis,50)
GN_best_dolphins, GN_mod_dolphins, GN_num_dolphins = comd_GN(dolphins,50)

In [5]:
from sklearn.cluster import SpectralClustering

# Spectral Clustering
def comd_SP(G,k):
    mG = np.array(nx.to_numpy_matrix(G))
    clustering = SpectralClustering(n_clusters=k,random_state=0, affinity='precomputed').fit(mG)
    clabels = clustering.labels_
    numc = clustering.n_clusters
    b = dict(enumerate(clabels,0)) # create a mapping of nodes indices to the community it belongs
    d = [] # a list of all communities (each list is a community)
    for i in range(0,numc):
        d.append([k for k,v in b.items() if float(v) == i])
    per = nx.algorithms.community.quality.performance(G,d)
    return d, per, numc

def find_best_SP_partition(G,k):
    com = []
    com_mod = []
    com_num = []
    for i in range(2,k):
        tcom,tcom_mod,tcom_num = comd_SP(G,i)
        com.append(tcom)
        com_mod.append(tcom_mod)
        com_num.append(tcom_num)
    index = com_mod.index(max(com_mod))
    G_SP = com[index]
    G_SP_mod = com_mod[index]
    G_SP_num = com_num[index]
    return G_SP, G_SP_mod, G_SP_num

football_SP, football_SP_mod, football_SP_num = find_best_SP_partition(football,50) 
lesmis_SP, lesmis_SP_mod, lesmis_SP_num = find_best_SP_partition(lesmis,50)  
dolphins_SP, dolphins_SP_mod, dolphins_SP_num = find_best_SP_partition(dolphins,50)

In [6]:
# Modularity Maximization
def comd_MM(G):
    c = list(nx.algorithms.community.modularity_max.greedy_modularity_communities(G))
    mm_G = []
    for i,j in enumerate(c):
        a = list(j)
        mm_G.append(a)
    per = nx.algorithms.community.quality.performance(G,mm_G)
    num_G = len(mm_G)
    return mm_G, per,num_G

football_MM, football_MM_mod, football_MM_num = comd_MM(football)
lesmis_MM, lesmis_MM_mod, lesmis_MM_num = comd_MM(lesmis)  
dolphins_MM, dolphins_MM_mod, dolphins_MM_num = comd_MM(dolphins)

### Compare best modularity scores for all algorithms

Παρακάτω, συγκρίνουμε τα scores των καλύτερων modularity που επιλέχθηκαν παραπάνω με εκείνα που προέκυψαν από την χρήση των κλασίκών αλγορίθμων της 2ης εργαστηριακής άσκησης.

In [12]:
print("Genetic algorithm results")
print("American College Football:\t", foot_modul)
print ("Les Miserables:\t\t\t", lesmis_modul)
print("Dolphins:\t\t\t", dolphins_modul)

Genetic algorithm results
American College Football:	 0.9067887109077041
Les Miserables:			 0.9036226930963773
Dolphins:			 0.9042834479111581


In [8]:
# Compare results 
print("American College Football")
print("Girvan-Newman: %r communities with modularity score %r" %(GN_num_football, GN_mod_football))
print("Spectral Clustering: %r communities with modularity score %r" %(football_SP_num, football_SP_mod))
print("Modularity Maximization: %r communities with modularity score %r" %(football_MM_num, football_MM_mod))

print()
print("Les Miserables")
print("Girvan-Newman: %r communities with modularity score %r" %(GN_num_lesmis, GN_mod_lesmis))
print("Spectral Clustering: %r communities with modularity score %r" %(lesmis_SP_num, lesmis_SP_mod))
print("Modularity Maximization: %r communities with modularity score %r" %(lesmis_MM_num, lesmis_MM_mod))

print()
print("Dolphins")
print("Girvan-Newman: %r communities with modularity score %r" %(GN_num_dolphins, GN_mod_dolphins))
print("Spectral Clustering: %r communities with modularity score %r" %(dolphins_SP_num, dolphins_SP_mod))
print("Modularity Maximization: %r communities with modularity score %r" %(dolphins_MM_num, dolphins_MM_mod))

American College Football
Girvan-Newman: 17 communities with modularity score 0.9581998474446987
Spectral Clustering: 15 communities with modularity score 0.9581998474446987
Modularity Maximization: 6 communities with modularity score 0.8681922196796339

Les Miserables
Girvan-Newman: 32 communities with modularity score 0.9630895420369104
Spectral Clustering: 17 communities with modularity score 0.9429254955570745
Modularity Maximization: 5 communities with modularity score 0.815105946684894

Dolphins
Girvan-Newman: 27 communities with modularity score 0.9439450026441036
Spectral Clustering: 26 communities with modularity score 0.9370703331570598
Modularity Maximization: 4 communities with modularity score 0.7424643046007403


#### Σχόλια για τα αποτελέσματα του γενετικού αλγόριθμου συγκριτικά με τους άλλους

Στην παραπάνω μελέτη έγινε χρήση γενετικών αλγορίθμων για εύρεση κοινοτήτων δοσμένων των γράφων, και τα αποτελέσματα που λάβαμε μετρώντας το performance ήταν πολύ ικανοποιητικά. Τα αποτελέσματα αυτά προκύπτουν, τις περισσότερες φορές με μεγάλες τιμές του $P_c$ και μικρές του $P_m$, ενώ πάντοτε επιλέγεται ο ίδιος ελιτισμός, σαν καλύτερη λύση, και συγκεκριμένα η τιμή 1. Επιπλέον, τα σκορ του γενετικού αλγορίθμου είναι πολύ κοντά στα σκορ των υπόλοιπων τεχνικών, με το SC και GN να είναι μονίμως λίγο καλύτερα του, ενώ το MM να είναι μονίμως αρκετά μακριά του. Αξίζει να σημειωθεί πως ο γενετικός αλγόριθμος δίνει και για τα τρία δοκιμαστικά γραφήματα παρά πολύ κοντινές επιδόσεις (90%), με αποτέλεσμα δεδομένης της καλής μοντελοποίησης των δεδομένων και της εύρεσης καλής συνάρτησης fitness, να αποδεικνύεται ιδιαίτερα σταθερός. Ακόμη, σημειώνουμε πως ο γενετικός αλγόριθμος λόγω της υψηλής του πολυπλοκότητας, που αυξάνεται εκθετικά με την αύξηση του αριθμού χρωμοσωμάτων, χρειάζεται περισσότερο χρόνο για να συγκλίνει ή να τερματισεί, σε σχέση με τους υπόλοιπους αλγορίθμους που χρησιμοποιήσαμε. Έτσι, καταλήγουμε πως οι γενετικοί αλγόριθμοι μπορούν να χρησιμοποιηθούν για την εύρεση κοινοτήτων δίνοντας πολύ καλές επιδόσεις, παρόμοιες με εκείνες των κλασικών αλγορίθμων.